In [1]:
!wandb login 86c64a2b2da45013d9c4f3c597d330492016a846

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import torchvision
import wandb

In [17]:
def create_model():
    model = torchvision.models.resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
    model.maxpool = nn.Identity()
    return model

In [ ]:
def create_model():
    model = torchvision.models.vit
    model.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
    model.maxpool = nn.Identity()
    return model

In [18]:
class CIFAR10Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, X):
        return self.model(X)

    def configure_optimizers(self):
        return torch.optim.RAdam(self.parameters())
        
    def evaluate(self, batch, stage):

        X, y = batch
        logits = self(X)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.functional.accuracy(preds, y, task='multiclass', num_classes=10)

        self.log(f"{stage}_loss", loss)
        self.log(f"{stage}_acc", accuracy)

        return loss, accuracy

    def training_step(self, batch, _):
        loss, _ = self.evaluate(batch, "train")
        return loss

    def validation_step(self, batch, _):
        self.evaluate(batch, "val")

    def test_step(self, batch, _):
        self.evaluate(batch, "test")

In [19]:
train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

test_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), cifar10_normalization()])

dm = CIFAR10DataModule(
    data_dir='CIFAR10/',
    batch_size=32,
    num_workers=4,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)

In [20]:
wandb.finish()
pl.seed_everything(0)
wandb_logger = WandbLogger()
trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu',
    devices=-1,
    logger=wandb_logger)
model = CIFAR10Model()

trainer.fit(model, dm)
wandb.finish()

Global seed set to 0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Epoch 29: 100%|████| 1563/1563 [02:07<00:00, 12.31it/s, loss=0.0877, v_num=ark3]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|████| 1563/1563 [02:07<00:00, 12.27it/s, loss=0.0877, v_num=ark3]


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_acc,▁▄▆▅▆▅▆▆▆▆█▆▆▇▇▇▇▇██▇█▇▇▇█▇██▇▇█████▇█▇█
train_loss,█▆▃▃▃▅▃▄▃▃▂▃▅▃▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▂▁▁▁▂▁▁▂▂▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▃▅▅▅▆▇▇▇▇▇▇▇▇▇▇██████████████
val_loss,█▆▄▄▃▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂
epoch,29
train_acc,1.0
train_loss,0.06236
trainer/global_step,37499
val_acc,0.9091


In [21]:
trainer.test(model, dm)

Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/service/server_sock.py", line 112, in run


Testing: 0it [00:00, ?it/s]

    shandler(sreq)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/service/server_sock.py", line 173, in server_record_publish
    iface = self._mux.get_stream(stream_id).interface
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/service/streams.py", line 199, in get_stream
    stream = self._streams[stream_id]
KeyError: '2y4cark3'


Testing DataLoader 0: 100%|███████████████████| 313/313 [00:07<00:00, 43.48it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8986999988555908
        test_loss           0.38874340057373047
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.38874340057373047, 'test_acc': 0.8986999988555908}]